In [1]:
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np
import random
import json

d:\Software\anaconda3\envs\pytorch_gpu\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
d:\Software\anaconda3\envs\pytorch_gpu\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback y

In [2]:
# Step 1: Load a small subset (1000 samples) of the C4 dataset in English
dataset = load_dataset("c4", "en", split="train", streaming=True)

C:\Users\80528\.cache\huggingface\modules\datasets_modules\datasets\c4\584d57ebe81c209b6c7f31727066d2c4b4bba37cb7092cdd83083d5ec11207db\c4.py:53: FutureWarning: Dataset 'c4' is deprecated and will be deleted. Use 'allenai/c4' instead.
  warnings.warn(


In [3]:
small_dataset = [x["text"] for _, x in zip(range(10000), dataset) if "text" in x]

In [29]:
# Step 2: Initialize a sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Efficient for semantic tasks

# Step 3: Compute embeddings for all sentences
embeddings = model.encode(small_dataset, show_progress_bar=True)

# Step 4: Calculate semantic differences
mean_embedding = np.mean(embeddings, axis=0)  # Average embedding for reference
semantic_differences = np.linalg.norm(embeddings - mean_embedding, axis=1)

# Step 5: Select top 10 samples with the largest semantic differences
top_indices = np.argsort(-semantic_differences)[:3]
top_samples = [small_dataset[i] for i in top_indices]

# Step 6: Write the top samples to a JSONL file
output_file = "3_samples.jsonl"
with open(output_file, "w") as f:
    for sample in top_samples:
        json.dump({"text": sample}, f)
        f.write("\n")  # JSONL format requires newline-separated JSON objects

print(f"Top samples have been stored in '{output_file}'.")

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Top samples have been stored in '3_samples.jsonl'.


In [4]:
# Step 2: Initialize a sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Efficient for semantic tasks

# Step 3: Compute embeddings for all sentences
embeddings = model.encode(small_dataset, show_progress_bar=True)

# Step 4: Calculate semantic differences
mean_embedding = np.mean(embeddings, axis=0)  # Average embedding for reference
semantic_differences = np.linalg.norm(embeddings - mean_embedding, axis=1)

# Step 5: Select top 3 samples with the largest semantic differences
top_indices = np.argsort(-semantic_differences)[:100]
top_samples = [{"id": int(i), "text": small_dataset[i]} for i in top_indices]

# Step 6: Write the top samples with IDs to a JSONL file
output_file = "100_samples.jsonl"
with open(output_file, "w") as f:
    for sample in top_samples:
        json.dump(sample, f)
        f.write("\n")  # JSONL format requires newline-separated JSON objects

print(f"Top samples with IDs have been stored in '{output_file}'.")

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Top samples with IDs have been stored in '100_samples.jsonl'.
